## Importing Libraries

In [1]:
import glob
import numpy as np
import random
import librosa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Dropout, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

## Loading Data

In [3]:
SEED = 2017
DATA_DIR = 'data/' 

In [4]:
files = glob.glob(DATA_DIR + "*.wav")
X_train, X_val = train_test_split(files, test_size=0.2, random_state=SEED)

In [6]:
print('Training examples: {}'.format(len(X_train)))
print('Validation examples: {}'.format(len(X_val)))

Training examples: 1920
Validation examples: 480


In [10]:
labels = []

for i in range(len(X_train)):
    label = X_train[i].split('/')[-1].split('_')[1]
    
    if label not in labels:
        labels.append(label)

In [11]:
print(labels)

['Vicki', 'Ralph', 'Alex', 'Bruce', 'Victoria', 'Tom', 'Fred', 'Albert', 'Agnes', 'Daniel', 'Princess', 'Junior', 'Steffi', 'Kathy', 'Samantha']


In [12]:
label_binarizer = LabelBinarizer()
label_binarizer.fit(list(set(labels)))

LabelBinarizer()

In [14]:
def one_hot_encode(label):
    return label_binarizer.transform(label)